# Feature Engineering

In [21]:
#Ładowanie bibliotek
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


import contractions
from symspellpy import SymSpell, Verbosity
from nrclex import NRCLex

from sklearn.feature_extraction.text import CountVectorizer
import gensim.downloader as api
import numpy as np
from tqdm.notebook import tqdm
import time
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder



In [2]:
#Podział danych na treningowe, testowe i walidacyjne

#df=pd.read_csv("YoutubeCommentsDataSet.csv")
#X = df.drop(columns=["Sentiment"])
#y = df['Sentiment']

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)
#y_test = pd.DataFrame(y_test, columns=["Sentiment"])


#ValidationData = pd.concat([X_val, y_val], axis=1)
#ValidationData.to_csv("ValidationData.csv", index=False)
#TestData = pd.concat([X_test, y_test], axis=1)
#TestData.to_csv("TestData.csv",index=False)
#TrainData = pd.concat([X_train, y_train], axis=1)
#TrainData.to_csv("TrainData.csv",index=False)


#print(f'Train shape: {X_train.shape}, Test shape: {X_test.shape}, Validation shape: {X_val.shape}')

In [5]:
class CleanCommentTransform(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        df['Comment'] = df['Comment'].fillna('')
        
        def preprocess_text(text):
            # Usuwanie interpunkcji
            text = text.lower()
            text = ''.join([char for char in text if char not in string.punctuation])
            
            # Tokenizacja
            tokens = word_tokenize(text)
            
            # Lematyzacja i usuwanie stopwords
            tokens = [self.lemmatizer.lemmatize(word) for word in tokens if word not in self.stop_words]
            
            return " ".join(tokens)
        
        # Zastosowanie transformacji na kolumnie 'Comment'
        df['Clean_Comment'] = df['Comment'].apply(preprocess_text)
        
        # Zwróć całą kolumnę 'Clean_Comment' jako lista tekstów
        return df['Clean_Comment'].values  # Teraz zwracasz numpy array, ale zawierający same teksty


In [7]:
class BoWwithNgramTransform(BaseEstimator, TransformerMixin):
    def __init__(self, ngram_range=(1, 2)):
        # Inicjalizacja CountVectorizer z ngramami (domyślnie 1-gram i 2-gram)
        self.vectorizer = CountVectorizer(ngram_range=ngram_range)
        
    def fit(self, X, y=None):
        # Dopasowanie wektoryzatora na danych
        # X jest teraz tablicą NumPy, więc musimy użyć odpowiedniego indeksowania
        self.vectorizer.fit(X.ravel())  # ravel() zamienia dane z numpy array na 1D
        return self
    
    def transform(self, X):
        # Przekształcenie tekstów na reprezentację BoW z ngramami
        # X jest teraz tablicą NumPy, więc musimy znowu przekształcić dane
        transformed = self.vectorizer.transform(X.ravel())  # ravel() zamienia dane z numpy array na 1D
        
        # Zwracamy dane w postaci macierzy NumPy
        return transformed.toarray()

In [9]:
class TextEmbedderTransform(BaseEstimator, TransformerMixin):
    def __init__(self, max_features=1000):
        # Możemy dodać opcję max_features, aby ograniczyć liczbę cech
        self.vectorizer = TfidfVectorizer(max_features=max_features)
        
    def fit(self, X, y=None):
        # Dostosowanie vectorizera do danych
        self.vectorizer.fit(X)
        return self
    
    def transform(self, X):
        # Transformacja danych na wektory
        return self.vectorizer.transform(X).toarray()

In [17]:
class CombinedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.bow = BoWwithNgramTransform()
        self.embed = TextEmbedderTransform()

    def fit(self, X, y=None):
        self.bow.fit(X)
        self.embed.fit(X)
        return self

    def transform(self, X):
        bow_features = self.bow.transform(X)
        embed_features = self.embed.transform(X)
        # Zakładamy, że oba zwracają macierze typu numpy / sparse
        from scipy.sparse import hstack, issparse
        if issparse(bow_features) or issparse(embed_features):
            return hstack([bow_features, embed_features])
        else:
            import numpy as np
            return np.hstack([bow_features, embed_features])

# Finalny pipeline
full_pipeline = Pipeline([
    ('clean_text', CleanCommentTransform()),
    ('features', CombinedFeatures())
])

In [19]:
# Przykładowy DataFrame z kolumną 'Comment'
df = pd.DataFrame({
    'Comment': [
        "This is the FIRST comment!!!",
        "Another one with some <html> tags & weird punctuation :)",
        None,  # testujemy też brak wartości
        "Final comment – with stuff to clean."
    ]
})

# Transformacja danych
X_transformed = full_pipeline.fit_transform(df)

# Sprawdzenie wyniku
print("Kształt cech:", X_transformed.shape)
print("Przykład pierwszego wektora cech:", X_transformed[0][:10])  # skrócone

Kształt cech: (4, 31)
Przykład pierwszego wektora cech: [0. 0. 0. 1. 0. 0. 0. 1. 1. 0.]
